# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import numpy as np
import pandas as pd
from IPython.display import display

import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use("seaborn-whitegrid")
colors = [x['color'] for x in plt.style.library['seaborn']['axes.prop_cycle']]

import dsttools

In [ ]:
# create folders if necessary
if not os.path.exists('figs'): os.makedirs('figs')    

# Settings

**Step 1:** Load useful information.

In [ ]:
datasets = dsttools.load_datasets_disk(datafolder='data')

**Step 2:** Choose settings.

In [ ]:
# a. globals
samples = ['p1','all'] # #  list of samples (from small to large!!!)
class par: pass # parameters
dfs = {} # dataframes
tbl = dsttools.SampleSelectionTable(obs_fac=12)

# b. ages
par.age_min = 35
par.age_max = 60

# c. years
par.bef_year = datasets['bfl']['years'][0]
par.years = np.arange(datasets['bfl']['years'][0],datasets['bfl']['years'][-1]+1)
par.T = par.years.size

# d. selection criteria
par.min_obs = 6         
par.rich_cutoff = 3_000
par.rich_cutoff_month = 500
par.self_employed_cutoff = 20
par.hours_cutoff = 0.95
par.wage_cutoff = 15
par.wage_cutoff_employed = 5   
par.full_time_share_cutoff = 0.5

# d. cohorts (implied)
par.cohorts = [par.years[0]-(par.age_max-par.min_obs+1),par.years[-1]-(par.age_min+(par.min_obs-1))]

**Step 3:** Show implications of settings.

In [ ]:
print(f'years: {par.years[0]}-{par.years[-1]}')       
print(f'cohorts: {par.cohorts[0]}-{par.cohorts[-1]}')

# Annual data

## Create

Load data from **population register** and **select on sex and cohorts**. Force the dataset to be **balanced**.


In [ ]:
def create(sample):
    
    # a. load bef
    bef = pd.read_parquet(f'data/befupd_{sample}.parquet')
    bef = bef[bef.year == par.bef_year+1]
        
    # b. birthyears
    birthyear = bef.birthday.dt.year
    
    # c. males and selected cohorts
    I = (bef.sex == 1) & (birthyear.between(par.cohorts[0],par.cohorts[-1]))
    birthyear = birthyear[I].values.astype('int')
    male_pnr = bef[I].pnr
    
    # d. check pnrs are unique
    assert male_pnr.is_unique
    par.N = male_pnr.size

    # e. create balanced dataset
    df = pd.DataFrame({
        'pnr':np.repeat(male_pnr,par.T),
        'year':np.tile(par.years,par.N),
        'birthyear':np.repeat(birthyear,par.T)
        }).set_index(['pnr','year'])
    
    # f. add age
    df['age'] = df.index.get_level_values('year')-df.birthyear
    
    # g. select on age
    I = df.age.between(par.age_min,par.age_max)
    df = df[I]
    dfs[('raw',sample)] = df
    
for sample in samples:
    %time create(sample)

display(dfs[('raw',sample)].reset_index().count())

tbl.add('Initial sample',dfs[('raw',sample)])
tbl.show()


**Inspect balancedness:**

In [ ]:
df = dfs[('raw',sample)]
pd.crosstab(df.birthyear,df.index.get_level_values('year'))

## Add income data

Load data from **annual income register** (various measures) and remove individuals with multiple observations.

In [ ]:
def add_annual_income_data(sample):
    
    # a. load ind
    ind = pd.read_parquet(f'data/indupd01_{sample}.parquet')
    ind = ind[['pnr','year','DISPON_13','PERINDKIALT_13','LOENMV_13','CORLOEN',
               'NETOVSKUD_13','PRIVAT_PENSION_13','OFFPENS_EFTERLON_13']]
        
    # b. find and remove duplicates
    ind['obs_per_year'] = ind.groupby(['pnr','year']).pnr.transform('count')
    ind['obs_per_year'] = ind.groupby('pnr')['obs_per_year'].transform('max')
        
    removed = ind[ind.obs_per_year > 1].pnr.unique().size
    print(f'Note: {removed} individuals removed due to multiple observations in a single year')
    
    # c. remove duplicates
    ind = ind[ind.obs_per_year == 1]
    ind = ind.set_index(['pnr','year'])
    
    # d. merge
    dfs[('ind',sample)] = dfs[('raw',sample)].join(ind,how='left').drop('obs_per_year',axis=1)

for sample in samples:
    %time add_annual_income_data(sample)

display(dfs[('ind',sample)].reset_index().count())

## Selection

The sample is selected using the following **criteria**:

1. Always in income register.
2. Never self-employed - NETOVSKUD_13 never above 20,000 DKK.
3. Never retired - never positive private (PRIVAT_PENSION_13) or public (OFFPENS_EFTERLON_13) pension.
4. Annual wage never more than 3 million DKK.
5. Monthly wage never more than 500,000 DKK.

**Scaling:** Monetary variables are scaled with average disposable income in our data set relative to 2018.

In [ ]:
def annual_select(sample):
    
    df = dfs[('ind',sample)]

    # a. missing information
    missing_info = df.CORLOEN.isna()
    no_missing_info = ~(missing_info.groupby('pnr').transform('max'))
    
    df = df[no_missing_info]
    tbl.add('Always in income register',df)

    # b. calculate scale
    df['scale'] = df.groupby('year').DISPON_13.transform('mean') / df.DISPON_13.xs(2018,level='year').mean()
    
    # c. remove self-employed
    self_employed = df.NETOVSKUD_13 > par.self_employed_cutoff*1_000*df['scale']
    never_self_employed = ~(self_employed.groupby('pnr').transform('max'))
    df = df[never_self_employed]
    tbl.add('Never self-employed',df)

    # d. remove retired
    retired = df.PRIVAT_PENSION_13 + df.OFFPENS_EFTERLON_13 > 0
    never_retired = ~(retired.groupby('pnr').transform('max'))
    
    df = df[never_retired]
    tbl.add('Never retired',df)   
    
    # e. clean and save
    df = df.rename(columns={'DISPON_13':'disp_inc',
                            'PERINDKIALT_13':'tot_inc',
                            'LOENMV_13':'wage_tot',
                            'CORLOEN':'wage_tot_alt'})
    
    df.disp_inc = df.disp_inc/1000
    df.tot_inc = df.tot_inc/1000
    df.wage_tot = df.wage_tot/1000
    df.wage_tot_alt = df.wage_tot_alt/1000
    
    drop = ['NETOVSKUD_13','PRIVAT_PENSION_13','OFFPENS_EFTERLON_13']
    dfs[('main_year',sample)] = df.drop(drop,axis=1)
    
for sample in samples:
    %time annual_select(sample)
    
display(dfs[('main_year',sample)].reset_index().count())
tbl.show()


# Monthly data

Load data from the **monthly income register** (broad, narrow, hours).

1. **Aggregate** to monthly level.
2. **Add annual data**
3. Create **balanced monthly sample**.

**Note:** Hours is relative to 160.33 hours ("full employment").

In [ ]:
def create_monthly(sample):
    
    # a. aggregation choices
    agg_dict = {'wage_broad':'sum','wage_narrow':'sum','hours':'sum'}
        
    # b. load
    bfl = pd.read_parquet(f'data/bfl_{sample}.parquet')
    bfl.wage_narrow = bfl.wage_narrow/1000
    bfl.wage_broad = bfl.wage_broad/1000
    
    # c. aggregate to monthly
    bfl['month'] = bfl.start_date.dt.month
    bfl_month = bfl.groupby(['pnr','year','month']).agg(agg_dict)
    
    assert np.all(bfl_month[['wage_broad','wage_narrow','hours']].notna())
    
    # d. create balanced monthly dataset
    pnrs = dfs[('main_year',sample)].index.unique('pnr')
    N = pnrs.size

    years = np.repeat(par.years,12)
    months = np.tile(np.arange(1,12+1),par.T)
    
    dates = [f'{year}{month}' for (year,month) in zip(years,months)]
    dates = pd.DatetimeIndex(pd.to_datetime(dates,format='%Y%m'))
              
    df = pd.DataFrame({'pnr':np.repeat(pnrs,par.T*12),
                       'year':np.tile(years,N),
                       'month':np.tile(months,N),
                       'date':np.tile(dates,N),                       
                      }).set_index(['pnr','year','month'])
     
    df = df.join(bfl_month,how='left')
    df = df.join(dfs[('main_year',sample)][['birthyear','age','scale']],how='inner',on=['pnr','year'])
        
    # e. find unemployed and fill with zeros
    df['unemployed'] = df.wage_narrow.isna() | (df.wage_narrow < par.wage_cutoff_employed*df.scale)    
    df.loc[df.wage_narrow.isna(),['wage_broad','wage_narrow','hours']] = 0
    
    assert np.all(bfl_month[['wage_broad','wage_narrow','hours']].notna())
    
    # f. find full-time employed
    enough_hours = df.hours > par.hours_cutoff
    enough_wage = df.wage_narrow > par.wage_cutoff*df.scale
    df['full_time'] = enough_hours & enough_wage

    # g. save
    dfs[('merged',sample)] = df
    
for sample in samples:
    %time create_monthly(sample)

display(dfs[('merged',sample)].reset_index().count())

**Full-time and unemployment:**

In [ ]:
pd.crosstab(dfs[('merged',sample)].full_time,dfs[('merged',sample)].unemployed)

**Add monthly data to annual data**:

In [ ]:
def add_monthly(sample):
    
    # a. aggregation choices
    agg_dict = {'wage_broad':'sum',
                'wage_narrow':'sum',
                'hours':'sum',
                'full_time':'mean'}
    
    # b. merge with annual data
    bfl_year = dfs[('merged',sample)].groupby(['pnr','year']).agg(agg_dict)  
    bfl_year_extra = dfs[('merged',sample)].groupby(['pnr','year']).agg({'wage_narrow':'max'}).rename(columns={'wage_narrow':'wage_narrow_max'})
    
    df = dfs[('main_year',sample)].join(bfl_year,how='left',on=['pnr','year']).join(bfl_year_extra,how='left',on=['pnr','year'])
    
    # c. drop rich - annual
    rich = df.wage_narrow > par.rich_cutoff*df.scale
    never_rich = ~(rich.groupby('pnr').transform('max'))
    
    df = df[never_rich]
    tbl.add('Annual wage never above 3 mil. DKK',df) 
    
    # d. drop rich - month
    rich = df.wage_narrow_max > par.rich_cutoff_month*df.scale
    never_rich = ~(rich.groupby('pnr').transform('max'))
    
    df = df[never_rich]
    tbl.add('Monthly wage never above 500,000 DKK',df) 
    
    # e. save
    dfs[('merged_year',sample)] = df

    # f. selected
    df = pd.DataFrame(dfs[('merged_year',sample)].index.unique(0)).set_index('pnr')
    dfs[('selected',sample)] = df.join(dfs[('merged',sample)])
    
for sample in samples:
    %time add_monthly(sample)

display(dfs[('merged_year',sample)].reset_index().count())
display(dfs[('selected',sample)].reset_index().count())
tbl.show()

# Selection on full-time employment share

In [ ]:
df = dfs[('selected',sample)]

**Histogram of hours:**

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
max_val = 1.5

# a. distribution
ax.hist(df.hours.clip(None,max_val),bins=500,cumulative=True,density=True)

# b. cutoff
ax.axvline(par.hours_cutoff,ls='--',lw=1,c='black',zorder=-1)

# c. save
ax.set_xlabel('hours relative to full-time')
ax.set_ylabel('cdf')
ax.set_xlim([0,max_val])
ax.set_ylim([0,1.0])
fig.tight_layout()
fig.savefig(f'figs/pre_hours_hist.pdf')

**Histogram of wages for those with enough hours:**

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
max_val = 150

# a. distribution
x = df.wage_narrow/df.scale
ax.hist(x[df.full_time].clip(None,max_val),bins=500,cumulative=True,density=True,label=None)

# b. cutoff
ax.axvline(par.wage_cutoff,ls='--',lw=1,c='black',zorder=-1)

# c. save
ax.set_xlabel('wage (scaled)')
ax.set_ylabel('cdf')
ax.set_xlim([0,max_val])
ax.set_ylim([0,1.0])

fig.tight_layout()
fig.savefig(f'figs/pre_wage_hist_enough_hours.pdf')

**Histogram of full-time:**

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

# a. distribution
x = df.full_time.groupby('pnr').mean()
ax.hist(x,bins=par.years.size*12,cumulative=True,density=True);

# b. cutoff
ax.axvline(par.full_time_share_cutoff,ls='--',lw=1,c='black',zorder=-1)

# c. save
ax.set_xlabel('share of months with full-time employement')
ax.set_ylabel('cdf') 
ax.set_xlim([0,1.0])
ax.set_ylim([0,1.0])

fig.tight_layout()
fig.savefig(f'figs/pre_full_time_hist.pdf')

**Select on full-time employed in 50 percent of months**:

In [ ]:
def full_time_employed(sample):
    
    # a. load annual data
    df = dfs[('merged_year',sample)]
              
    # b. select
    keep = df.full_time.groupby('pnr').transform('mean') > par.full_time_share_cutoff    
    keep.name = 'keep'
    dfs[('final_year',sample)]= df[keep].drop('full_time',axis=1)

    tbl.add('Full-time employed 50 percent of the time',dfs[('final_year',sample)])
        
    # c. load and drop monthly data
    df = dfs[('selected',sample)].join(keep.groupby('pnr').first(),how='left',on='pnr')
    dfs[('final',sample)] = df[df.keep].drop('keep',axis=1)
    df = dfs[('final',sample)]

for sample in samples:
    %time full_time_employed(sample)

display(dfs[('final_year',sample)].reset_index().count())
display(dfs[('final',sample)].reset_index().count())
    
tbl.show()

**Share of months with full-time employment:**

In [ ]:
print(f'full-time share {dfs[("final",sample)].full_time.mean():.3}')
print(f'unemployed share {dfs[("final",sample)].unemployed.mean():.3f}')

# Overview

## Annual

In [ ]:
merged_year = dfs[('merged_year',sample)]
display(merged_year.groupby('year').agg('mean').round(2))
display(pd.crosstab(merged_year.birthyear,merged_year.index.get_level_values('year')))

In [ ]:
merged_year.isnull().mean()

## Monthly

In [ ]:
merged = dfs[('merged',sample)]
display(merged.groupby('year').agg('mean').round(3))
display(pd.crosstab(merged.birthyear,merged.index.get_level_values('year')))

In [ ]:
merged.isnull().mean()

# Comparisons

In [ ]:
merged_year[['wage_broad','wage_narrow','wage_tot','wage_tot_alt','tot_inc','disp_inc']].corr()

In [ ]:
# a. difference
diff = merged_year.wage_narrow-merged_year.wage_tot
abs_diff = np.abs(diff)
rel_abs_diff = abs_diff/merged_year.wage_narrow

x = np.linspace(0.5,1.00,51)

# b. differences
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
diff_cap = np.clip(diff,-100,100)
ax.hist(diff_cap,bins=500,density=True,cumulative=True)
ax.set_xlabel('DKK')
ax.set_ylabel('cdf')
ax.set_title('differences')

# b. absolute differences
fig = plt.figure()
ax = fig.add_subplot(1,1,1)
y = abs_diff.quantile(x)
ax.plot(x[:-2],y.values[:-2])
ax.set_xlabel('percentile')
ax.set_ylabel('DKK')
ax.set_title('absolute difference')

# c. relative absolute differences
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

y_all = rel_abs_diff.quantile(x)
ax.plot(x[:-2],y_all.values[:-2],label='all')

I = merged_year.wage_narrow > 10*merged_year.scale 
y_sel = rel_abs_diff[I].quantile(x)
ax.plot(x[:-2],y_sel.values[:-2],label='wage > 10,000')

ax.set_xlabel('percentile')
ax.set_ylabel('percent')
ax.set_title('relative absolute difference')
ax.legend();

In [ ]:
# a. difference by year
diff = merged_year.wage_narrow-merged_year.wage_tot
abs_diff = np.abs(diff)
rel_abs_diff = abs_diff/merged_year.wage_narrow

# b. quantiles by year
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

x = np.linspace(0.7,0.98,51)
ys = rel_abs_diff.groupby(['year']).quantile(x)

for year in [2011,2017,2018]:
    y = ys.xs(year,level='year')
    ax.plot(x,y,label=str(year))

ax.set_xlabel('percentile')
ax.set_ylabel('percent')
ax.set_title('relative absolute difference')
ax.legend();

**Varying cutoffs in DKK and percent:**

1. **ok:** Discrepancy less than DKK cutoff *or* less than percent cutoff.
2. **keep:** Always ok

In [ ]:
df = pd.DataFrame()
for abs_diff_cut in [5,10,20,30,40,50]:
    for rel_abs_diff_cut in [0.025,0.05,0.075,0.10,np.inf]:
        
        ok_or = (abs_diff < abs_diff_cut) | (rel_abs_diff < rel_abs_diff_cut)
        ok_and = (abs_diff < abs_diff_cut) & (rel_abs_diff < rel_abs_diff_cut)
        keep = ok_or.groupby('pnr').transform('min')
        
        row = {'DKK':[abs_diff_cut],
               'percent':[rel_abs_diff_cut],
               'ok_or':[ok_or.mean()],
               'ok_and':[ok_and.mean()],
               'keep':[keep.mean()]}
        df = df.append(pd.DataFrame(row))
        
display(df)

# Save

In [ ]:
for sample in samples:
    %time dfs[('final_year',sample)].to_parquet(f'data/final_year_{sample}.parquet')
    %time dfs[('final',sample)].to_parquet(f'data/final_{sample}.parquet')

**Write sample selection table in Latex:**

In [ ]:
tbl.show()
tbl.latex('figs/sample_selection.tex')